In [34]:
import pickle
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [55]:
games = pd.read_pickle('game_data/games_four_years.pkl')
g2018 = pd.read_pickle('game_data/games_2018.pkl')

In [57]:
# g2018.info()

In [4]:
'''Shuffle DataFrame'''
games = games.sample(frac=1).reset_index(drop=True)

In [5]:
Xy = games[['W', 'Wp', 'ppg', 'pApg', 'FGp', '3Pp', 'FTp', 'ORBpg', 'RBpg', 
            'ASTpg', 'STLpg', 'BLKpg', 'TOpg', 'PFpg', 'sos', 'OPppg', 
            'OPpApg', 'OPFGp', 'OP3Pp', 'OPFTp', 'OPORBpg', 'OPRBpg', 
            'OPASTpg', 'OPSTLpg', 'OPBLKpg', 'OPTOpg', 'OPPFpg', 'OPsos']]

In [6]:
# Set up features and targets
X = Xy.iloc[:, 1:]
y = Xy.iloc[:, 0]

In [7]:
'''Train test split'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [8]:
'''Standardize Data'''
scale = StandardScaler()
scale.fit(X_train)
X_train = scale.transform(X_train)
X_test = scale.transform(X_test)

In [9]:
'''Fit model on training data'''
lg = LogisticRegression()
lg.fit(X_train, y_train)
lg_predict = lg.predict(X_test)

In [59]:
lg_accuracy = metrics.accuracy_score(y_test, lg_predict)
lg_precision = metrics.precision_score(y_test, lg_predict)
lg_recall = metrics.recall_score(y_test, lg_predict)
lg_f1 = metrics.f1_score(y_test, lg_predict)
print('Accuracy: {:.2f} (% predicted correctly)'.format(lg_accuracy))
print('Precision: {:.2f} (predicted positives % correct)'.format(lg_precision))
print('Recall: {:.2f} (% of positives predicted correctly)'.format(lg_recall))
print('f1 Score: {:.2f} (weighted average of Precision and Recall)'.format(lg_f1))

Accuracy: 0.79 (% predicted correctly)
Precision: 0.78 (predicted positives % correct)
Recall: 0.79 (% of positives predicted correctly)
f1 Score: 0.79 (weighted average of Precision and Recall)


In [33]:
np.isnan(y_train)[np.isnan(y_train) == False].size

3664

In [25]:
def cross_val_logistic(X, y, model, n_folds=5, random_seed=154):
    """Estimate the in- and out-of-sample error of a model using cross
    validation.
 
    Parameters
    ----------
 
    X: np.array
      Matrix of predictors, standardized
 
    y: np.array
      Target array, standardized
 
    model: sklearn model object.
      The estimator to fit.  Must have fit and predict methods.
 
    n_folds: int
      The number of folds in the cross validation.
 
    random_seed: int
      A seed for the random number generator, for repeatability.
 
    Returns
    -------
    mean accuracy across folds
    """
    kf = KFold(n_folds)
    accuracy_list = []
    cfs = []
    for train_index,test_index in kf.split(X):
        # define variables
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]
        print(y_test)
        # fit model
#         model.fit(X_train, y_train)
#         y_hat_train = model.predict(X_train)
#         y_hat_test = model.predict(X_test) 
#         # evaluate model
#         accuracy_test = metrics.accuracy_score(y_test, y_hat_test)
#         errorlist.append(accuracy_test)
#         # store coefficients
#         cfs.append (model.coef_)
#     # select best coefficients 
#     accuracies = np.asarray(accuracy_list)
#     idx_max_test_accuracy = accuracies.argmax()   
#     cfs_best = cfs[idx_min_test_accuracy]
    
#     return(errors)

In [53]:
logcv = LogisticRegression()
ridgecv = Ridge(alpha=.5)
lassocv = Lasso(alpha=.5)
enetcv = ElasticNet(alpha=.5)
log_cv_acc = cross_validate(logcv, X_train, y_train, scoring='accuracy', cv=5, return_train_score=False)
# ridge_cv_acc = cross_validate(ridgecv, X_train, y_train, scoring='accuracy', cv=5, return_train_score=False)
# lasso_cv_acc = cross_validate(lassocv, X_train, y_train, scoring='accuracy', cv=5, return_train_score=False)
# enet_cv_acc = cross_validate(enetcv, X_train, y_train, scoring='accuracy', cv=5, return_train_score=False)
print(log_cv_acc['test_score'].mean())
# print(ridge_cv_acc['test_score'].mean())

0.786570862186


In [ ]:
model = Ridge(alpha=.5)
X = X_train
y = y_train
cv_accuracy = cross_val_logistic(X, y, model)